In [ ]:

import torch.nn.init as init

from src.unet.unet_models import UNet
from src.models import VTFPredictor

In [ ]:
model = VTFPredictor(model_name="FPathPredictor", loss_name="SketchMaskLoss")

In [ ]:
import torch
import torch.nn as nn

fc1 = nn.Linear(21, 1, bias=False)
init.constant_(fc1.weight.data, 0.1)
print(fc1.weight.data.shape)

In [ ]:
import torch
import torch.nn as nn

fc1 = nn.Linear(21, 1, bias=False)
init.constant_(fc1.weight.data, 0.1)
print(fc1.weight.data.shape) 

infodraw = torch.ones(1, 1, 3, 3)
infodraw[0, 0, 2, 0] = 0.7
infodraw[0, 0, 2, 1] = 0.8
infodraw[0, 0, 2, 2] = 0.9

threshold = 0.99
mask = infodraw < threshold

fpath = torch.ones(1, 21, 3, 3) / 21
fpath_perm = fpath.permute(0, 2, 3, 1)
y_hat_perm = fc1(fpath_perm)
y_hat = y_hat_perm.permute(0, 3, 1, 2)

masked_y_hat = mask * y_hat
masked_infodraw = ~mask * infodraw

result = masked_y_hat + masked_infodraw

print(f"fpath       : {fpath.shape}")
print(f"fpath_perm  : {fpath_perm.shape}, After permute(0, 2, 3, 1)")
print(f"y_hat_perm  : {y_hat_perm.shape}, After nn.Linear(21, 1)")
print(f"y_hat       : {y_hat.shape}, After permute(0, 3, 1, 2)")

print(f"infodraw        : \n{infodraw}")
print(f"mask            : \n{mask}")
print(f"~mask           : \n{~mask}")
print(f"masked_y_hat    : \n{masked_y_hat}")
print(f"masked_infodraw : \n{masked_infodraw}")
print(f"result          : \n{result}")

In [ ]:
fpath[0, :, 0, 0]

In [ ]:
fc1.weight.data

In [ ]:
y_hat

In [ ]:
import torch
import torch.nn as nn

fc1 = nn.Linear(21, 1, bias=False)
init.constant_(fc1.weight.data, 0.1)
print(fc1.weight.data.shape)

infodraw = torch.ones(1, 1, 3, 3)
infodraw[0, 0, 2, 0] = 0.7
infodraw[0, 0, 2, 1] = 0.8
infodraw[0, 0, 2, 2] = 0.9

threshold = 0.99

B, _, H, W = infodraw.shape
result = infodraw.clone()
fpath = torch.ones(1, 21, 3, 3) / 21
fpath_list = []
for b in range(B):
    for h in range(H):
        for w in range(W):
            if infodraw[b, :, h, w].item() < threshold:
                # print((infodraw[b, :, h, w], (h, w)))
                fpath_list.append((fpath[b, :, h, w], (h, w)))  

    for fpath_vec, (h, w) in fpath_list:
        y_hat = fc1(fpath_vec.unsqueeze(0))
        result[b, 0, h, w] = y_hat

print(f"result: \n{result}")

# fpath_perm = fpath.permute(0, 2, 3, 1)
# y_hat_perm = fc1(fpath_perm)
# y_hat = y_hat_perm.permute(0, 3, 1, 2)

# masked_y_hat = mask * y_hat
# masked_infodraw = ~mask * infodraw

# result = masked_y_hat + masked_infodraw

# print(f"fpath       : {fpath.shape}")
# print(f"fpath_perm  : {fpath_perm.shape}, After permute(0, 2, 3, 1)")
# print(f"y_hat_perm  : {y_hat_perm.shape}, After nn.Linear(21, 1)")
# print(f"y_hat       : {y_hat.shape}, After permute(0, 3, 1, 2)")

# print(f"infodraw        : \n{infodraw}")
# print(f"mask            : \n{mask}")
# print(f"~mask           : \n{~mask}")
# print(f"masked_y_hat    : \n{masked_y_hat}")
# print(f"masked_infodraw : \n{masked_infodraw}")
# print(f"result          : \n{result}")

In [ ]:
y_hat = torch.tensor([[[
    [1.0, 0.6, 0.1],
    [0.5, 0.0, 0.9],
    [0.2, 0.7, 1.0]
]]])
y = torch.tensor([[[
    [1, 1, 0],
    [1, 0, 1],
    [0, 1, 1]
]]])
print(f"y_hat   : {y_hat.shape}")
print(f"y       : {y.shape}")

In [ ]:
loss_per_pixel = -(y * torch.log(y_hat+1e-8) + (1-y) * torch.log(1-y_hat+1e-8))
mask = torch.tensor([[[
    [0, 0, 1],
    [0, 1, 0],
    [1, 0, 0]
]]])

masked_loss_per_pixel = mask * loss_per_pixel
loss = torch.sum(masked_loss_per_pixel) / torch.sum(mask)


print(f"loss per pixel          : \n{loss_per_pixel}")
print(f"masked loss per pixel   : \n{masked_loss_per_pixel}")
print(f"loss mean               : {loss}")

In [ ]:
from src.preprocesses import VTFPreprocessor, ImagePreprocessor, InfodrawPreprocessor, TargetPreprocessor

In [ ]:
target = TargetPreprocessor.get("/home/work/joono/VTFSketch/dataset/simple_data/test/targets/line_901.png")
infodraw = InfodrawPreprocessor.get("/home/work/joono/VTFSketch/dataset/simple_data/test/infodraws/color_901_out.png")
vtf = VTFPreprocessor.get("/home/work/joono/VTFSketch/dataset/simple_data/test/vtfs/color_901_fpath_of_infodraw.npz")

target = torch.tensor(target).unsqueeze(0)
infodraw = torch.tensor(infodraw).unsqueeze(0)
vtf = torch.tensor(vtf).unsqueeze(0)

In [ ]:
target.shape, infodraw.shape, vtf.shape

In [ ]:
region_W, region_B = target, (1-target)

torch.set_printoptions(precision=2)
print(f"region_W: \n{region_W[0, 0, 200:210, 200:210]}")
print(f"region_B: \n{region_B[0, 0, 200:210, 200:210]}")
print(f"infodraw: \n{infodraw[0, 0, 200:210, 200:210]}")

In [ ]:
mask = (infodraw < 0.99).float()
print(f"mask    : \n{mask[0, 0, 200:210, 200:210]}")

In [ ]:
print(f"white pixels: {torch.sum(mask * target)}")
print(f"Black pixels: {torch.sum(mask * (1-target))}")

In [ ]:
mask_mul_target = mask * target
print(f"mask * target: \n{mask_mul_target[0, 0, 200:210, 200:210]}")

In [ ]:
mask_mul_sketch = mask * (1-target)
print(f"mask * 1-target: \n{mask_mul_sketch[0, 0, 200:210, 200:210]}")

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
np_infodraw = np.array((mask * target).squeeze())
print(np_infodraw.shape)
np_infodraw = np.stack([np_infodraw, np_infodraw, np_infodraw], axis=2)
print(np_infodraw.shape)
cv2.rectangle(np_infodraw, (200, 200), (210, 210), color=(0, 0, 255), thickness=2)
plt.imshow(np_infodraw[100:400, 100:400])
plt.axis('off')

In [ ]:
model = VTFPredictor.load_from_checkpoint("/home/work/joono/VTFSketch/VTFPredictor/rmyfvx6h/checkpoints/best-checkpoint-val_f1score=0.5868685245513916.ckpt").to('cpu')

In [ ]:
img = ImagePreprocessor.get("/home/work/joono/VTFSketch/dataset/simple_data/test/imgs/color_901.png")
y_hat = model(vtf=vtf, infodraw=infodraw, img=img)

In [ ]:
y_hat[0, 0, 200:210, 200:210]

In [ ]:
loss_per_pixel = -1 * (target * torch.log(y_hat) + (1-target) * torch.log(1-y_hat))

In [ ]:
loss_per_pixel[0, 0, 200:210, 200:210]
loss = torch.sum(loss_per_pixel[0, 0, 200:210, 200:210]) / 100

In [ ]:
print(f"loss_per_pixel: \n{loss_per_pixel[0, 0, 200:210, 200:210]}")
print(f"loss          : {torch.sum(loss_per_pixel[0, 0, 200:210, 200:210]) / torch.sum(mask[0, 0, 200:210, 200:210])}")

In [ ]:
mask_W = mask * target
print(f"mask_W: \n{mask_W[0, 0, 200:210, 200:210]}")
mask_W_loss_per_pixel = mask_W * loss_per_pixel
print(f"mask_W_loss_per_pixel: \n{mask_W_loss_per_pixel[0, 0, 200:210, 200:210]}")
print(f"mask_W_loss         : {torch.sum(mask_W_loss_per_pixel) / torch.sum(mask_W)}")
print(f"mask_W_loss_w_mask  : {torch.sum(mask_W_loss_per_pixel) / torch.sum(mask)}")

In [ ]:
mask_B = mask * (1-target)
print(f"mask_B: \n{mask_B[0, 0, 200:210, 200:210]}")
mask_B_loss_per_pixel = mask_B * loss_per_pixel
print(f"mask_B_loss_per_pixel: \n{mask_B_loss_per_pixel[0, 0, 200:210, 200:210]}")
print(f"mask_B_loss         : {torch.sum(mask_B_loss_per_pixel) / torch.sum(mask_B)}")
print(f"mask_B_loss_w_mask  : {torch.sum(mask_B_loss_per_pixel) / torch.sum(mask)}")

In [ ]:
loss_per_pixel 

In [ ]:
np_infodraw = np.array(y_hat.squeeze().detach())
print(np_infodraw.shape)
np_infodraw = np.stack([np_infodraw, np_infodraw, np_infodraw], axis=2)
print(np_infodraw.shape)
cv2.rectangle(np_infodraw, (200, 200), (210, 210), color=(0, 0, 255), thickness=1)
plt.imshow(np_infodraw[100:400, 100:400])
plt.axis('off')

In [1]:
import torch
from src.models import MinFPathPredictor

In [2]:
model = MinFPathPredictor().to('cuda')

In [3]:
vtf = torch.rand(2, 21, 1024, 1024).to("cuda")
infodraw = torch.rand(2, 1, 1024, 1024).to("cuda")
img = torch.rand(2, 3, 1024, 1024).to("cuda")

y_hat = model(vtf=vtf, img=img, infodraw=infodraw)

  3%|▎         | 33476/1038113 [00:08<04:11, 3986.72it/s]


KeyboardInterrupt: 